In [2]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.show(1)

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eb3b1464-267b-4b21-8be2-bbc221ccf151;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 66ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

+--------------------+---+----+---+-----------+
|             user_id|  x|   y|  t|pixel_color|
+--------------------+---+----+---+-----------+
|vKtmwRQwAYtw5poO4...|804|1992|  0|    #FF3881|
+--------------------+---+----+---+-----------+
only showing top 1 row



<div style="color:pink">

Idee: 
-Daten nach Nutzern aggregieren und die Zeit zwischen aufeinanderfolgenden Pixeln vergleichen
-Zeit (Differenz) festhalten und mit dem nächsten Zeitraum vergleichen
-Falls die Differenz abweicht, ist der Nutzer wahrscheinlich kein Bot(?)

</div>

In [15]:
from src.features.feature_functions import group_dataframe_by_user
groupedFrame = group_dataframe_by_user(dataFrame)
groupedFrame.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+-------------------------+
|             user_id|count|     collect_list(x)|     collect_list(y)|     collect_list(t)|collect_list(pixel_color)|
+--------------------+-----+--------------------+--------------------+--------------------+-------------------------+
|AxBTmhBMbr1sbDw/2...|   20|[1403, 777, 1741,...|[2, 8, 10, 20, 19...|[2344, 2344, 2344...|     [#FFA800, #FFA800...|
|DIECtVqoJqBlL6A0C...|   15|[903, 1650, 936, ...|[1850, 227, 1861,...|[2710, 2408, 293,...|     [#FFFFFF, #FF4500...|
|VmiTfdqG3P1MlZokx...|   15|[906, 913, 947, 1...|[1865, 1871, 1868...|[2767, 2164, 2466...|     [#000000, #FFFFFF...|
|FjHp8Fg+uuheibUWk...|   15|[998, 998, 998, 9...|[1466, 1466, 1466...|[60, 420, 420, 22...|     [#000000, #000000...|
|X8gpDfO6+Z87SKKTy...|   15|[919, 1642, 914, ...|[1858, 214, 1836,...|[2331, 2637, 212,...|     [#51E9F4, #000000...|
+--------------------+-----+--------------------+-------

In [14]:
import pyspark.sql.functions as F
#Nutzung von dataFrame oder groupedFrame sinnvoller?
selfjoinedFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
selfjoinedFrame = selfjoinedFrame.where('df2.t > df1.t') #nur vorläufig, hat nichts mit dem Ergebnis zu tun^^
selfjoinedFrame = selfjoinedFrame.select(F.col('df1.user_id'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'))
selfjoinedFrame.show(2)

+--------------------+----+----+
|             user_id|  t1|  t2|
+--------------------+----+----+
|++0nCmngtZ68vbg6p...| 734|2690|
|++3l0k1HSXKiAsHkH...|1058|1381|
+--------------------+----+----+
only showing top 2 rows



<div style ="color:pink">
-Differenz speichern und beim nächsten Vergleich erneut verwenden?
-Nutzer, die das Kriterium bei einem der Vergleiche nicht erfüllen, müssten nicht erneut betrachtet werden(?) 
</div>